## 과제 요약
- Case 9의 마지막 에포크에서 val accuracy가 0.6018을 기록하여, 과제 수행에 성공하였다. Val Accuracy를 높였던 결정적인 요소로는 
> - 4 Layer로 이루어진 Convolutional Block을 4개까지 직렬로 쌓는 것(그 이상의 Block은 유의미한 영향이 없었다)
> - Adam Optimizer가 아닌 Momentum 계열 Optimizer를 사용하는 것(Adaptive Gradient 계열을 사용하면 학습이 이루어지지 않는 버그가 있었다)
> - Convolution Filter의 수가 64 -> 128 -> 256 -> 512 순으로 늘어나 풍부한 수의 feature를 확보하는 것(filter 수가 6 -> 16 -> 32 -> 64 에 불과하면 정확도 43%를 벗어나지 못했다)<br><br>
> 를 들 수 있다.

In [4]:
import pandas as pd
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms


# 코드 다시 돌리기 위한 seed 고정
import random
import numpy as np
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
device

device(type='cuda')

In [6]:
train_transform = transforms.Compose([    
    transforms.AutoAugment(transforms.autoaugment.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),
])        

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])    

train = torchvision.datasets.CIFAR100(root="./", train=True, download=True, transform=train_transform)
test = torchvision.datasets.CIFAR100(root="./", train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=256,
                                           shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=256,
                                          shuffle=False, num_workers=2)
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class MyModel(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(MyModel, self).__init__()
        """Base Model"""
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 100)
        
        

    def forward(self, x):
        """Base Model"""
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = MyModel(3, 100).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

model


MyModel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=100, bias=True)
)

In [ ]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model.state_dict()

train epoch: 1----------------


100%|██████████| 196/196 [00:14<00:00, 13.52it/s]



train_loss :  4.527656900639436
val_acc :  0.0707
train epoch: 2----------------


100%|██████████| 196/196 [00:14<00:00, 13.65it/s]



train_loss :  4.233913866841063
val_acc :  0.1173
train epoch: 3----------------


...


train_loss :  3.747718228369343
val_acc :  0.181
train epoch: 9----------------


100%|██████████| 196/196 [00:14<00:00, 13.50it/s]



train_loss :  3.714530322016502
val_acc :  0.1924
train epoch: 10----------------


 25%|██▌       | 49/196 [00:05<00:17,  8.21it/s]


KeyboardInterrupt: 

# Case1. 
- Conv 계층을 5계층으로 하여 깊이를 더한다.
- Channel 수는 3 -> 64 -> 64 -> 32 -> 16 -> 16 로 하여 더 많은 feature map을 뽑아내겠다.
- Kernel의 크기는 처음 3계층 3 x 3, 마지막 2계층 2 x 2로 하여 국소적 맥락에만 집중하도록 하겠다.
- image size는 32 -> 16 -> 8 -> 4 -> 2
- FC Node는 36 -> 180 -> 140 -> 100
- Optimizer Adam, learning rate 0.01

In [ ]:
class Case1Model(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(Case1Model, self).__init__()
        """Base Model"""
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.pool1 = nn.MaxPool2d(2, 2, 1)
        
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2, 1)

        self.conv3 = nn.Conv2d(64, 32, 3)
        self.pool3 = nn.MaxPool2d(2, 2, 1)

        self.conv4 = nn.Conv2d(32, 16, 2)
        self.pool4 = nn.MaxPool2d(2, 2, 1)

        self.conv5 = nn.Conv2d(16, 16, 2)
        self.pool5 = nn.MaxPool2d(2, 1, 1)
        
        self.fc1 = nn.Linear(16 * 2 * 2, 180)
        self.fc2 = nn.Linear(180, 140)
        self.fc3 = nn.Linear(140, 100)
        
        

    def forward(self, x):
        """Base Model"""
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = self.pool5(F.relu(self.conv5(x)))

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model1 = Case1Model(3, 100).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

model1


In [ ]:
for epoch in range(10):
    model1.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model1.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model1(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model1.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model1(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model1.state_dict()

# Case 2
- 앞선 Case 1에선 Gradient Vanishing이 일어난 것으로 보인다.
- 이에 BatchNormalization으로 이를 해소하고, Learning Rate도 0.1로 다시 높이겠다.

In [ ]:
class Case2Model(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(Case2Model, self).__init__()
        """Base Model"""
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.pool1 = nn.MaxPool2d(2, 2, 1)
        self.batchNorm1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2, 1)
        self.batchNorm2 = nn.BatchNorm2d(64)


        self.conv3 = nn.Conv2d(64, 32, 3)
        self.pool3 = nn.MaxPool2d(2, 2, 1)
        self.batchNorm3 = nn.BatchNorm2d(32)

        self.conv4 = nn.Conv2d(32, 16, 2)
        self.pool4 = nn.MaxPool2d(2, 2, 1)
        self.batchNorm4 = nn.BatchNorm2d(16)


        self.conv5 = nn.Conv2d(16, 16, 2)
        self.pool5 = nn.MaxPool2d(2, 1, 1)

        
        self.fc1 = nn.Linear(16 * 2 * 2, 180)
        self.fc2 = nn.Linear(180, 140)
        self.fc3 = nn.Linear(140, 100)
        
        

    def forward(self, x):
        """Base Model"""
        x1 = self.batchNorm1(self.pool1(F.relu(self.conv1(x))))
        x2 = self.batchNorm2(self.pool2(F.relu(self.conv2(x1))))
        x3 = self.batchNorm3(self.pool3(F.relu(self.conv3(x2))))
        x4 = self.batchNorm4(self.pool4(F.relu(self.conv4(x3))))
        x5 = self.pool5(F.relu(self.conv5(x4)))

        x = torch.flatten(x5, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model2 = Case2Model(3, 100).to(device)
optimizer = optim.Adam(model2.parameters(), lr=0.1)

model2


Case2Model(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (batchNorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (batchNorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (batchNorm3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (batchNorm4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d

In [ ]:
for epoch in range(10):
    model2.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model2.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model2(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model2.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model2(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model2.state_dict()

train epoch: 1----------------


100%|██████████| 196/196 [00:36<00:00,  5.37it/s]



train_loss :  4.499614194947846
val_acc :  0.0273
train epoch: 2----------------


100%|██████████| 196/196 [00:38<00:00,  5.15it/s]



train_loss :  4.066124045118993
val_acc :  0.0573
train epoch: 3----------------


 37%|███▋      | 73/196 [00:16<00:27,  4.50it/s]


KeyboardInterrupt: 

# Case 3
- Optimizer를 원래의 SGD로 바꾸어보겠다.

In [ ]:
class Case3Model(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(Case2Model, self).__init__()
        """Base Model"""
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.pool1 = nn.MaxPool2d(2, 2, 1)
        self.batchNorm1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2, 1)
        self.batchNorm2 = nn.BatchNorm2d(64)


        self.conv3 = nn.Conv2d(64, 32, 3)
        self.pool3 = nn.MaxPool2d(2, 2, 1)
        self.batchNorm3 = nn.BatchNorm2d(32)

        self.conv4 = nn.Conv2d(32, 16, 2)
        self.pool4 = nn.MaxPool2d(2, 2, 1)
        self.batchNorm4 = nn.BatchNorm2d(16)


        self.conv5 = nn.Conv2d(16, 16, 2)
        self.pool5 = nn.MaxPool2d(2, 1, 1)

        
        self.fc1 = nn.Linear(16 * 2 * 2, 180)
        self.fc2 = nn.Linear(180, 140)
        self.fc3 = nn.Linear(140, 100)
        
        

    def forward(self, x):
        """Base Model"""
        x1 = self.batchNorm1(self.pool1(F.relu(self.conv1(x))))
        x2 = self.batchNorm2(self.pool2(F.relu(self.conv2(x1))))
        x3 = self.batchNorm3(self.pool3(F.relu(self.conv3(x2))))
        x4 = self.batchNorm4(self.pool4(F.relu(self.conv4(x3))))
        x5 = self.pool5(F.relu(self.conv5(x4)))

        x = torch.flatten(x5, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model3 = Case3Model(3, 100).to(device)
optimizer = optim.SGD(model3.parameters(), lr=0.1)

model3


MyModel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=100, bias=True)
)

In [ ]:
for epoch in range(10):
    model3.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model3.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model3(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model3.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model3(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model3.state_dict()

train epoch: 1----------------


  0%|          | 0/196 [00:03<?, ?it/s]


RuntimeError: running_mean should contain 256 elements not 64

## Case 4
- Residual Connection을 추가한다.
- Optimizer를 Nesterov Momentum으로 변경한다.

In [ ]:
class Case4Model(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(Case4Model, self).__init__()
        """Base Model"""
        self.conv1 = nn.Conv2d(3, 6, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.batchNorm1 = nn.BatchNorm2d(6)
        #6 * (16 * 16)

        self.conv2 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm2 = nn.BatchNorm2d(6)
        self.conv3 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm3 = nn.BatchNorm2d(6)
        #6 * (16 * 16)
        
        self.conv4 = nn.Conv2d(6, 16, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.batchNorm4 = nn.BatchNorm2d(16)
        #16 * (8 * 8)

        self.conv5 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm6 = nn.BatchNorm2d(16)
        #16 * (8 * 8)
        
        self.fc1 = nn.Linear(1024, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 100)
        
        

    def forward(self, x):
        """Base Model"""
        x1 = self.batchNorm1(self.pool1(F.relu(self.conv1(x))))
        
        x2 = self.batchNorm2(F.relu(self.conv2(x1)))
        x3 = self.batchNorm3(F.relu(F.relu(self.conv3(x2)) + x1))

        x4 = self.batchNorm4(self.pool2(F.relu(self.conv4(x3))))

        x5 = self.batchNorm5(F.relu(self.conv5(x4)))
        x6 = self.batchNorm6(F.relu(F.relu(self.conv6(x5)) + x4))

        x = torch.flatten(x6, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model4 = Case4Model(3, 100).to(device)
optimizer = optim.SGD(model4.parameters(), momentum=0.9, nesterov=True, lr=0.1)

model4


Case4Model(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batchNorm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm3): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batchNorm4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm5): BatchNorm2d(16, eps=1e-05, momentum=0.1, 

In [ ]:
for epoch in range(10):
    model4.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model4.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model4(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model4.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model4(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model4.state_dict()

train epoch: 1----------------


100%|██████████| 196/196 [00:19<00:00,  9.81it/s]



train_loss :  3.5816236235657515
val_acc :  0.2176
train epoch: 2----------------


100%|██████████| 196/196 [00:20<00:00,  9.78it/s]



train_loss :  2.7541073645864214
val_acc :  0.2833
train epoch: 3----------------


...


train_loss :  1.393361126281777
val_acc :  0.3473
train epoch: 9----------------


100%|██████████| 196/196 [00:19<00:00,  9.83it/s]



train_loss :  1.2683219334908895
val_acc :  0.3244
train epoch: 10----------------


  0%|          | 0/196 [00:01<?, ?it/s]


KeyboardInterrupt: 

## Case 6
- Layer를 더 deep하게 쌓겠다.

In [ ]:
class Case5Model(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(Case5Model, self).__init__()
        """Base Model"""
        self.conv1_0 = nn.Conv2d(3, 6, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.batchNorm1_0 = nn.BatchNorm2d(6)
        #6 * (16 * 16)

        self.conv1_1 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_1 = nn.BatchNorm2d(6)
        self.conv1_2 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_2 = nn.BatchNorm2d(6)
        self.conv1_3 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_3 = nn.BatchNorm2d(6)
        self.conv1_4 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_4 = nn.BatchNorm2d(6)
        #6 * (16 * 16)
        
        self.conv2_0 = nn.Conv2d(6, 16, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.batchNorm2_0 = nn.BatchNorm2d(16)
        #16 * (8 * 8)

        self.conv2_1 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_1 = nn.BatchNorm2d(16)
        self.conv2_2 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_2 = nn.BatchNorm2d(16)
        self.conv2_3 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_3 = nn.BatchNorm2d(16)
        self.conv2_4 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_4 = nn.BatchNorm2d(16)
        #16 * (8 * 8)
        
        self.conv3_0 = nn.Conv2d(16, 32, 3, 1, 1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.batchNorm3_0 = nn.BatchNorm2d(32)
        #32 * (4 * 4)

        self.conv3_1 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_1 = nn.BatchNorm2d(32)
        self.conv3_2 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_2 = nn.BatchNorm2d(32)
        self.conv3_3 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_3 = nn.BatchNorm2d(32)
        self.conv3_4 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_4 = nn.BatchNorm2d(32)
        #32 * (4 * 4)

        # self.conv4_0 = nn.Conv2d(32, 64, 3, 1, 1)
        # self.pool4 = nn.MaxPool2d(2, 2)
        # self.batchNorm4_0 = nn.BatchNorm2d(64)
        # #64 * (2 * 2)

        # self.conv4_1 = nn.Conv2d(64, 64, 3, 1, 1)
        # self.batchNorm4_1 = nn.BatchNorm2d(64)
        # self.conv4_2 = nn.Conv2d(64, 64, 3, 1, 1)
        # self.batchNorm4_2 = nn.BatchNorm2d(64)
        # self.conv4_3 = nn.Conv2d(64, 64, 3, 1, 1)
        # self.batchNorm4_3 = nn.BatchNorm2d(64)
        # self.conv4_4 = nn.Conv2d(64, 64, 3, 1, 1)
        # self.batchNorm4_4 = nn.BatchNorm2d(64)

        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 100)
        
        

    def forward(self, x):
        """Base Model"""
        x1_0 = self.batchNorm1_0(self.pool1(F.relu(self.conv1_0(x))))
        x1_1 = self.batchNorm1_1(F.relu(self.conv1_1(x1_0)))
        x1_2 = self.batchNorm1_2(F.relu(self.conv1_2(x1_1)) + x1_0)
        x1_3 = self.batchNorm1_3(F.relu(self.conv1_3(x1_2)) + x1_0 + x1_1)
        x1_4 = self.batchNorm1_4(F.relu(self.conv1_4(x1_3)) + x1_0 + x1_1 + x1_2)

        x2_0 = self.batchNorm2_0(self.pool2(F.relu(self.conv2_0(x1_4))))
        x2_1 = self.batchNorm2_1(F.relu(self.conv2_1(x2_0)))
        x2_2 = self.batchNorm2_2(F.relu(self.conv2_2(x2_1)) + x2_0)
        x2_3 = self.batchNorm2_3(F.relu(self.conv2_3(x2_2)) + x2_0 + x2_1)
        x2_4 = self.batchNorm2_4(F.relu(self.conv2_4(x2_3)) + x2_0 + x2_1 + x2_2)


        x3_0 = self.batchNorm3_0(self.pool3(F.relu(self.conv3_0(x2_4))))
        x3_1 = self.batchNorm3_1(F.relu(self.conv3_1(x3_0)))
        x3_2 = self.batchNorm3_2(F.relu(self.conv3_2(x3_1)) + x3_0)
        x3_3 = self.batchNorm3_3(F.relu(self.conv3_3(x3_2)) + x3_0 + x3_1)
        x3_4 = self.batchNorm3_4(F.relu(self.conv3_4(x3_3)) + x3_0 + x3_1 + x3_2)

        x = torch.flatten(x3_4, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model5 = Case5Model(3, 100).to(device)
optimizer = optim.Adam(model5.parameters(), momentum=0.9, nesterov=True, lr=0.1)

model5


Case5Model(
  (conv1_0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batchNorm1_0): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_1): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm1_1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_2): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm1_2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_3): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm1_3): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_4): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm1_4): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_0): Conv2d(6, 16, kerne

In [ ]:
for epoch in range(50):
    model5.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model5.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model5(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model5.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model5(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model5.state_dict()

train epoch: 1----------------


100%|██████████| 196/196 [00:34<00:00,  5.74it/s]



train_loss :  3.592560725552695
val_acc :  0.2312
train epoch: 2----------------


100%|██████████| 196/196 [00:42<00:00,  4.66it/s]



train_loss :  2.759140508515494
val_acc :  0.3129
train epoch: 3----------------


...

100%|██████████| 196/196 [01:29<00:00,  2.19it/s]



train_loss :  0.5449669713876686
val_acc :  0.3566
train epoch: 29----------------


100%|██████████| 196/196 [01:30<00:00,  2.18it/s]



train_loss :  0.5189416296019846
val_acc :  0.3454
train epoch: 30----------------


  0%|          | 0/196 [00:00<?, ?it/s]

## Case 7
- Linear Layer를 더 Deep하게 쌓아보겠다.
- DropOut을 적용한다.

In [ ]:
class Case7Model(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(Case7Model, self).__init__()
        """Base Model"""
        self.conv1_0 = nn.Conv2d(3, 6, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.batchNorm1_0 = nn.BatchNorm2d(6)
        #6 * (16 * 16)

        self.conv1_1 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_1 = nn.BatchNorm2d(6)
        self.conv1_2 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_2 = nn.BatchNorm2d(6)
        self.conv1_3 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_3 = nn.BatchNorm2d(6)
        self.conv1_4 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_4 = nn.BatchNorm2d(6)
        #6 * (16 * 16)
        
        self.conv2_0 = nn.Conv2d(6, 16, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.batchNorm2_0 = nn.BatchNorm2d(16)
        #16 * (8 * 8)

        self.conv2_1 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_1 = nn.BatchNorm2d(16)
        self.conv2_2 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_2 = nn.BatchNorm2d(16)
        self.conv2_3 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_3 = nn.BatchNorm2d(16)
        self.conv2_4 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_4 = nn.BatchNorm2d(16)
        #16 * (8 * 8)
        
        self.conv3_0 = nn.Conv2d(16, 32, 3, 1, 1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.batchNorm3_0 = nn.BatchNorm2d(32)
        #32 * (4 * 4)

        self.conv3_1 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_1 = nn.BatchNorm2d(32)
        self.conv3_2 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_2 = nn.BatchNorm2d(32)
        self.conv3_3 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_3 = nn.BatchNorm2d(32)
        self.conv3_4 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_4 = nn.BatchNorm2d(32)
        #32 * (4 * 4)

        self.conv4_0 = nn.Conv2d(32, 128, 3, 1, 1)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.batchNorm4_0 = nn.BatchNorm2d(128)
        #128 * (2 * 2)

        self.conv4_1 = nn.Conv2d(128, 128, 3, 1, 1)
        self.batchNorm4_1 = nn.BatchNorm2d(128)
        self.conv4_2 = nn.Conv2d(128, 128, 3, 1, 1)
        self.batchNorm4_2 = nn.BatchNorm2d(128)
        self.conv4_3 = nn.Conv2d(128, 128, 3, 1, 1)
        self.batchNorm4_3 = nn.BatchNorm2d(128)
        self.conv4_4 = nn.Conv2d(128, 128, 3, 1, 1)
        self.batchNorm4_4 = nn.BatchNorm2d(128)
        #128 * (2 * 2)

        self.fc1 = nn.Linear(512, 256)
        self.batchNormL_1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 256)
        self.batchNormL_2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.batchNormL_3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 128)
        self.batchNormL_4 = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, 100)
        
        self.dropout = nn.Dropout(0.2)        

    def forward(self, x):
        """Base Model"""
        x1_0 = self.batchNorm1_0(self.pool1(F.relu(self.conv1_0(x))))
        x1_1 = self.batchNorm1_1(F.relu(self.conv1_1(x1_0)))
        x1_2 = self.batchNorm1_2(F.relu(self.conv1_2(x1_1)) + x1_0)
        x1_3 = self.batchNorm1_3(F.relu(self.conv1_3(x1_2)) + x1_0 + x1_1)
        x1_4 = self.batchNorm1_4(F.relu(self.conv1_4(x1_3)) + x1_0 + x1_1 + x1_2)

        x2_0 = self.batchNorm2_0(self.pool2(F.relu(self.conv2_0(x1_4))))
        x2_1 = self.batchNorm2_1(F.relu(self.conv2_1(x2_0)))
        x2_2 = self.batchNorm2_2(F.relu(self.conv2_2(x2_1)) + x2_0)
        x2_3 = self.batchNorm2_3(F.relu(self.conv2_3(x2_2)) + x2_0 + x2_1)
        x2_4 = self.batchNorm2_4(F.relu(self.conv2_4(x2_3)) + x2_0 + x2_1 + x2_2)

        x3_0 = self.batchNorm3_0(self.pool3(F.relu(self.conv3_0(x2_4))))
        x3_1 = self.batchNorm3_1(F.relu(self.conv3_1(x3_0)))
        x3_2 = self.batchNorm3_2(F.relu(self.conv3_2(x3_1)) + x3_0)
        x3_3 = self.batchNorm3_3(F.relu(self.conv3_3(x3_2)) + x3_0 + x3_1)
        x3_4 = self.batchNorm3_4(F.relu(self.conv3_4(x3_3)) + x3_0 + x3_1 + x3_2)

        x4_0 = self.batchNorm4_0(self.pool4(F.relu(self.conv4_0(x3_4))))
        x4_1 = self.batchNorm4_1(F.relu(self.conv4_1(x4_0)))
        x4_2 = self.batchNorm4_2(F.relu(self.conv4_2(x4_1)) + x4_0)
        x4_3 = self.batchNorm4_3(F.relu(self.conv4_3(x4_2)) + x4_0 + x4_1)
        x4_4 = self.batchNorm4_4(F.relu(self.conv4_4(x4_3)) + x4_0 + x4_1 + x4_2)

        x = torch.flatten(x4_4, 1) # flatten all dimensions except batch
        x = self.dropout(x)
        x = self.batchNormL_1(F.relu(self.fc1(x)))
        x = self.dropout(x)
        x = self.batchNormL_2(F.relu(self.fc2(x)))
        x = self.dropout(x)
        x = self.batchNormL_3(F.relu(self.fc3(x)))
        x = self.dropout(x)
        x = self.batchNormL_4(F.relu(self.fc4(x)))
        x = self.dropout(x)
        x = self.fc5(x)
        return x

model7 = Case7Model(3, 100).to(device)
optimizer = optim.SGD(model7.parameters(), momentum=0.9, nesterov=True, lr=0.05)


In [ ]:
for epoch in range(50):
    model7.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model7.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model7(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model7.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model7(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model7.state_dict()

train epoch: 1----------------


100%|██████████| 196/196 [00:37<00:00,  5.21it/s]


train_loss :  4.369052942918271


val_acc :  0.0863
train epoch: 2----------------


100%|██████████| 196/196 [00:33<00:00,  5.82it/s]

...


train_loss :  2.013939844710486


val_acc :  0.4358
train epoch: 50----------------


100%|██████████| 196/196 [00:33<00:00,  5.91it/s]


train_loss :  1.9825612093721117


val_acc :  0.442


## Case 8
- BottleNeck 구조를 만들겠다.

In [ ]:
class Case8Model(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(Case8Model, self).__init__()
        """Base Model"""
        self.conv1_0 = nn.Conv2d(3, 6, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.batchNorm1_0 = nn.BatchNorm2d(6)
        #6 * (16 * 16)

        self.conv1_1 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_1 = nn.BatchNorm2d(6)
        self.conv1_2 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_2 = nn.BatchNorm2d(6)
        self.conv1_3 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_3 = nn.BatchNorm2d(6)
        self.conv1_4 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_4 = nn.BatchNorm2d(6)
        #6 * (16 * 16)
        
        self.conv1_5 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_5 = nn.BatchNorm2d(6)
        self.conv1_6 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_6 = nn.BatchNorm2d(6)
        self.conv1_7 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_7 = nn.BatchNorm2d(6)
        self.conv1_8 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_8 = nn.BatchNorm2d(6)
        #6 * (16 * 16)

        self.conv1_9 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_9 = nn.BatchNorm2d(6)
        self.conv1_10 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_10 = nn.BatchNorm2d(6)
        self.conv1_11 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_11 = nn.BatchNorm2d(6)
        self.conv1_12 = nn.Conv2d(6, 6, 3, 1, 1)
        self.batchNorm1_12 = nn.BatchNorm2d(6)
        #6 * (16 * 16)

        self.conv2_0 = nn.Conv2d(6, 16, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.batchNorm2_0 = nn.BatchNorm2d(16)
        #16 * (8 * 8)

        self.conv2_1 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_1 = nn.BatchNorm2d(16)
        self.conv2_2 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_2 = nn.BatchNorm2d(16)
        self.conv2_3 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_3 = nn.BatchNorm2d(16)
        self.conv2_4 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_4 = nn.BatchNorm2d(16)
        #16 * (8 * 8)

        self.conv2_5 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_5 = nn.BatchNorm2d(16)
        self.conv2_6 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_6 = nn.BatchNorm2d(16)
        self.conv2_7 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_7 = nn.BatchNorm2d(16)
        self.conv2_8 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_8 = nn.BatchNorm2d(16)
        #16 * (8 * 8)

        self.conv2_9 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_9 = nn.BatchNorm2d(16)
        self.conv2_10 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_10 = nn.BatchNorm2d(16)
        self.conv2_11 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_11 = nn.BatchNorm2d(16)
        self.conv2_12 = nn.Conv2d(16, 16, 3, 1, 1)
        self.batchNorm2_12 = nn.BatchNorm2d(16)
        #16 * (8 * 8)


        self.conv3_0 = nn.Conv2d(16, 32, 3, 1, 1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.batchNorm3_0 = nn.BatchNorm2d(32)
        #32 * (4 * 4)

        self.conv3_1 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_1 = nn.BatchNorm2d(32)
        self.conv3_2 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_2 = nn.BatchNorm2d(32)
        self.conv3_3 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_3 = nn.BatchNorm2d(32)
        self.conv3_4 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_4 = nn.BatchNorm2d(32)
        #32 * (4 * 4)

        self.conv3_5 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_5 = nn.BatchNorm2d(32)
        self.conv3_6 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_6 = nn.BatchNorm2d(32)
        self.conv3_7 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_7 = nn.BatchNorm2d(32)
        self.conv3_8 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_8 = nn.BatchNorm2d(32)
        #32 * (4 * 4)

        self.conv3_9 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_9 = nn.BatchNorm2d(32)
        self.conv3_10 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_10 = nn.BatchNorm2d(32)
        self.conv3_11 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_11 = nn.BatchNorm2d(32)
        self.conv3_12 = nn.Conv2d(32, 32, 3, 1, 1)
        self.batchNorm3_12 = nn.BatchNorm2d(32)
        #32 * (4 * 4)

        self.fc1 = nn.Linear(512, 256)
        self.batchNormL_1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 256)
        self.batchNormL_2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.batchNormL_3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 128)
        self.batchNormL_4 = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, 100)
        
        self.dropout = nn.Dropout(0.2)        

    def forward(self, x):
        """Base Model"""
        x1_0 = self.batchNorm1_0(self.pool1(F.relu(self.conv1_0(x))))
        
        x1_1 = self.batchNorm1_1(F.relu(self.conv1_1(x1_0)))
        x1_2 = self.batchNorm1_2(F.relu(self.conv1_2(x1_1)) + x1_0)
        x1_3 = self.batchNorm1_3(F.relu(self.conv1_3(x1_2)) + x1_0 + x1_1)
        x1_4 = self.batchNorm1_4(F.relu(self.conv1_4(x1_3)) + x1_0 + x1_1 + x1_2)

        x1_5 = self.batchNorm1_1(F.relu(self.conv1_5(x1_0)))
        x1_6 = self.batchNorm1_2(F.relu(self.conv1_6(x1_5)) + x1_0)
        x1_7 = self.batchNorm1_3(F.relu(self.conv1_7(x1_6)) + x1_0 + x1_5)
        x1_8 = self.batchNorm1_4(F.relu(self.conv1_8(x1_7)) + x1_0 + x1_5 + x1_6)

        x1_9 = self.batchNorm1_1(F.relu(self.conv1_9(x1_0)))
        x1_10 = self.batchNorm1_2(F.relu(self.conv1_10(x1_9)) + x1_0)
        x1_11 = self.batchNorm1_3(F.relu(self.conv1_11(x1_10)) + x1_0 + x1_9)
        x1_12 = self.batchNorm1_4(F.relu(self.conv1_12(x1_11)) + x1_0 + x1_9 + x1_10)

        x1 = x1_4 + x1_8 + x1_12
        x2_0 = self.batchNorm2_0(self.pool2(F.relu(self.conv2_0(x1))))

        x2_1 = self.batchNorm2_1(F.relu(self.conv2_1(x2_0)))
        x2_2 = self.batchNorm2_2(F.relu(self.conv2_2(x2_1)) + x2_0)
        x2_3 = self.batchNorm2_3(F.relu(self.conv2_3(x2_2)) + x2_0 + x2_1)
        x2_4 = self.batchNorm2_4(F.relu(self.conv2_4(x2_3)) + x2_0 + x2_1 + x2_2)

        x2_5 = self.batchNorm2_5(F.relu(self.conv2_1(x2_0)))
        x2_6 = self.batchNorm2_6(F.relu(self.conv2_2(x2_5)) + x2_0)
        x2_7 = self.batchNorm2_7(F.relu(self.conv2_3(x2_6)) + x2_0 + x2_5)
        x2_8 = self.batchNorm2_8(F.relu(self.conv2_4(x2_7)) + x2_0 + x2_5 + x2_6)

        x2_9 = self.batchNorm2_9(F.relu(self.conv2_1(x2_0)))
        x2_10 = self.batchNorm2_10(F.relu(self.conv2_2(x2_9)) + x2_0)
        x2_11 = self.batchNorm2_11(F.relu(self.conv2_3(x2_10)) + x2_0 + x2_9)
        x2_12 = self.batchNorm2_12(F.relu(self.conv2_4(x2_11)) + x2_0 + x2_9 + x2_10)

        x2 = x2_4 + x2_8 + x2_12
        x3_0 = self.batchNorm3_0(self.pool3(F.relu(self.conv3_0(x2))))

        x3_1 = self.batchNorm3_1(F.relu(self.conv3_1(x3_0)))
        x3_2 = self.batchNorm3_2(F.relu(self.conv3_2(x3_1)) + x3_0)
        x3_3 = self.batchNorm3_3(F.relu(self.conv3_3(x3_2)) + x3_0 + x3_1)
        x3_4 = self.batchNorm3_4(F.relu(self.conv3_4(x3_3)) + x3_0 + x3_1 + x3_2)

        x3_5 = self.batchNorm3_5(F.relu(self.conv3_1(x3_0)))
        x3_6 = self.batchNorm3_6(F.relu(self.conv3_2(x3_5)) + x3_0)
        x3_7 = self.batchNorm3_7(F.relu(self.conv3_3(x3_6)) + x3_0 + x3_5)
        x3_8 = self.batchNorm3_8(F.relu(self.conv3_4(x3_7)) + x3_0 + x3_5 + x3_6)

        x3_9 = self.batchNorm3_9(F.relu(self.conv3_1(x3_0)))
        x3_10 = self.batchNorm3_10(F.relu(self.conv3_2(x3_9)) + x3_0)
        x3_11 = self.batchNorm3_11(F.relu(self.conv3_3(x3_10)) + x3_0 + x3_9)
        x3_12 = self.batchNorm3_12(F.relu(self.conv3_4(x3_11)) + x3_0 + x3_9 + x3_10)
        
        x4 = x3_4 + x3_8 + x3_12

        x = torch.flatten(x4, 1) # flatten all dimensions except batch
        x = self.dropout(x)
        x = self.batchNormL_1(F.relu(self.fc1(x)))
        x = self.dropout(x)
        x = self.batchNormL_2(F.relu(self.fc2(x)))
        x = self.dropout(x)
        x = self.batchNormL_3(F.relu(self.fc3(x)))
        x = self.dropout(x)
        x = self.batchNormL_4(F.relu(self.fc4(x)))
        x = self.dropout(x)
        x = self.fc5(x)
        return x

model8 = Case8Model(3, 100).to(device)
optimizer = optim.SGD(model8.parameters(), momentum=0.9, nesterov=True, lr=0.1)


In [ ]:
for epoch in range(100):
    model8.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model8.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model8(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model8.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model8(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model8.state_dict()

train epoch: 1----------------


100%|██████████| 196/196 [01:48<00:00,  1.81it/s]



train_loss :  3.940081620702938
val_acc :  0.1493
train epoch: 2----------------


100%|██████████| 196/196 [01:36<00:00,  2.02it/s]



train_loss :  3.4054188253928204
val_acc :  0.2157
train epoch: 3----------------


...


train_loss :  1.3515896213297942
val_acc :  0.4292
train epoch: 68----------------


100%|██████████| 196/196 [01:18<00:00,  2.50it/s]



train_loss :  1.3456984843526567
val_acc :  0.4362
train epoch: 69----------------


  8%|▊         | 15/196 [00:09<01:55,  1.56it/s]


KeyboardInterrupt: 

In [7]:
class Case9Model(nn.Module):
    #Cifar100 Shape: [Batch, Channel(3), 32, 32]
    def __init__(self, in_channels, num_classes):
        super(Case9Model, self).__init__()
        """Base Model"""
        self.conv1_0 = nn.Conv2d(3, 64, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.batchNorm1_0 = nn.BatchNorm2d(64)
        #64 * (16 * 16)

        self.conv1_1 = nn.Conv2d(64, 64, 3, 1, 1)
        self.batchNorm1_1 = nn.BatchNorm2d(64)
        self.conv1_2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.batchNorm1_2 = nn.BatchNorm2d(64)
        self.conv1_3 = nn.Conv2d(64, 64, 3, 1, 1)
        self.batchNorm1_3 = nn.BatchNorm2d(64)
        self.conv1_4 = nn.Conv2d(64, 64, 3, 1, 1)
        self.batchNorm1_4 = nn.BatchNorm2d(64)
        #64 * (16 * 16)
        
        self.conv2_0 = nn.Conv2d(64, 128, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.batchNorm2_0 = nn.BatchNorm2d(128)
        #128 * (8 * 8)

        self.conv2_1 = nn.Conv2d(128, 128, 3, 1, 1)
        self.batchNorm2_1 = nn.BatchNorm2d(128)
        self.conv2_2 = nn.Conv2d(128, 128, 3, 1, 1)
        self.batchNorm2_2 = nn.BatchNorm2d(128)
        self.conv2_3 = nn.Conv2d(128, 128, 3, 1, 1)
        self.batchNorm2_3 = nn.BatchNorm2d(128)
        self.conv2_4 = nn.Conv2d(128, 128, 3, 1, 1)
        self.batchNorm2_4 = nn.BatchNorm2d(128)
        #128 * (8 * 8)
        
        self.conv3_0 = nn.Conv2d(128, 256, 3, 1, 1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.batchNorm3_0 = nn.BatchNorm2d(256)
        #256 * (4 * 4)

        self.conv3_1 = nn.Conv2d(256, 256, 3, 1, 1)
        self.batchNorm3_1 = nn.BatchNorm2d(256)
        self.conv3_2 = nn.Conv2d(256, 256, 3, 1, 1)
        self.batchNorm3_2 = nn.BatchNorm2d(256)
        self.conv3_3 = nn.Conv2d(256, 256, 3, 1, 1)
        self.batchNorm3_3 = nn.BatchNorm2d(256)
        self.conv3_4 = nn.Conv2d(256, 256, 3, 1, 1)
        self.batchNorm3_4 = nn.BatchNorm2d(256)
        #256 * (4 * 4)

        self.conv4_0 = nn.Conv2d(256, 512, 3, 1, 1)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.batchNorm4_0 = nn.BatchNorm2d(512)
        #512 * (2 * 2)

        self.conv4_1 = nn.Conv2d(512, 512, 3, 1, 1)
        self.batchNorm4_1 = nn.BatchNorm2d(512)
        self.conv4_2 = nn.Conv2d(512, 512, 3, 1, 1)
        self.batchNorm4_2 = nn.BatchNorm2d(512)
        self.conv4_3 = nn.Conv2d(512, 512, 3, 1, 1)
        self.batchNorm4_3 = nn.BatchNorm2d(512)
        self.conv4_4 = nn.Conv2d(512, 512, 3, 1, 1)
        self.batchNorm4_4 = nn.BatchNorm2d(512)
        #512 * (2 * 2)

        self.conv5_0 = nn.Conv2d(512, 512, 2)
        self.batchNorm5_0 = nn.BatchNorm2d(512)
        #512 * (1 * 1)

        
        self.fc1 = nn.Linear(512, 128)
        self.batchNormL_1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 100)
        
        self.dropout = nn.Dropout(0.2)        

    def forward(self, x):
        """Base Model"""
        x1_0 = self.batchNorm1_0(self.pool1(F.relu(self.conv1_0(x))))
        x1_1 = self.batchNorm1_1(F.relu(self.conv1_1(x1_0)))
        x1_2 = self.batchNorm1_2(F.relu(self.conv1_2(x1_1)) + x1_0)
        x1_3 = self.batchNorm1_3(F.relu(self.conv1_3(x1_2)) + x1_0 + x1_1)
        x1_4 = self.batchNorm1_4(F.relu(self.conv1_4(x1_3)) + x1_0 + x1_1 + x1_2)

        x2_0 = self.batchNorm2_0(self.pool2(F.relu(self.conv2_0(x1_4))))
        x2_1 = self.batchNorm2_1(F.relu(self.conv2_1(x2_0)))
        x2_2 = self.batchNorm2_2(F.relu(self.conv2_2(x2_1)) + x2_0)
        x2_3 = self.batchNorm2_3(F.relu(self.conv2_3(x2_2)) + x2_0 + x2_1)
        x2_4 = self.batchNorm2_4(F.relu(self.conv2_4(x2_3)) + x2_0 + x2_1 + x2_2)

        x3_0 = self.batchNorm3_0(self.pool3(F.relu(self.conv3_0(x2_4))))
        x3_1 = self.batchNorm3_1(F.relu(self.conv3_1(x3_0)))
        x3_2 = self.batchNorm3_2(F.relu(self.conv3_2(x3_1)) + x3_0)
        x3_3 = self.batchNorm3_3(F.relu(self.conv3_3(x3_2)) + x3_0 + x3_1)
        x3_4 = self.batchNorm3_4(F.relu(self.conv3_4(x3_3)) + x3_0 + x3_1 + x3_2)

        x4_0 = self.batchNorm4_0(self.pool4(F.relu(self.conv4_0(x3_4))))

        x4_1 = self.batchNorm4_1(F.relu(self.conv4_1(x4_0)))
        x4_2 = self.batchNorm4_2(F.relu(self.conv4_2(x4_1)) + x4_0)
        x4_3 = self.batchNorm4_3(F.relu(self.conv4_3(x4_2)) + x4_0 + x4_1)
        x4_4 = self.batchNorm4_4(F.relu(self.conv4_4(x4_3)) + x4_0 + x4_1 + x4_2)

        x5 = self.batchNorm5_0(F.relu(self.conv5_0(x4_4)))

        x = torch.flatten(x5, 1) # flatten all dimensions except batch
        x = self.dropout(x)
        x = self.batchNormL_1(F.relu(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model9 = Case9Model(3, 100).to(device)
optimizer = optim.SGD(model9.parameters(), momentum=0.9, nesterov=True, lr=0.05)


In [8]:
for epoch in range(100):
    model9.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model9.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model9(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model9.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model9(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model9.state_dict()

train epoch: 1----------------


100%|██████████| 196/196 [00:44<00:00,  4.39it/s]


train_loss :  3.987913574491228


val_acc :  0.1985
train epoch: 2----------------


100%|██████████| 196/196 [00:39<00:00,  4.93it/s]

...

100%|██████████| 196/196 [00:39<00:00,  4.94it/s]


train_loss :  0.3763592916027624


val_acc :  0.6018
train epoch: 71----------------


 38%|███▊      | 74/196 [00:14<00:23,  5.26it/s]


KeyboardInterrupt: ignored